##### Hypothese :
location_id est la clé primaire de la table location, qui est basée sur l'identification de maniere unique d'une localisation, sous plusieurs données, et dans notre cas, on a que le departement de residence, on considerera qu'elle est la location_id, car il n'est pas necessaire de créer une table location, qui n'aura pas toute les données

Au travers des joinctures, il n'existerait pas de localisation connues pour ces etablissements, objectivement parlant, une valeur par defaut est imputée 

In [31]:
import sqlite3
import pandas as pd

#Charger les données necessaires
etablissement_info = pd.read_csv("t_mcoaae.csv")
prestations_remboursees = pd.read_csv("er_prs_f.csv")
assurance_info = pd.read_csv("ir_ben_r.csv")

#créer les colonnes necessaires
care_site = pd.DataFrame()
care_site['cc_site_id'] = range(1, len(etablissement_info) +1)
care_site['care_site_name'] = etablissement_info['soc_rai']
care_site['care_site_source_value'] = etablissement_info['eta_num']
care_site

#Faire les joinctures pour recuperer la location_id
prestations_remboursees['etb_pre_fin'] = prestations_remboursees['etb_pre_fin'].fillna('0')
prestations_remboursees['etb_pre_fin'] = prestations_remboursees['etb_pre_fin'].astype(int)
table = pd.merge(prestations_remboursees, etablissement_info, left_on='etb_pre_fin', right_on='eta_num', how='inner')
table1 = pd.merge(table, assurance_info, on='NUM_ENQ', how='inner')

#Aucune valeur ne correspond, on va imputer une valeur par defaut à location_id
care_site['location_id'] = care_site.get('location_id', 0)

#SQLite
conn = sqlite3.connect("omop.db")
cursor = conn.cursor()

cursor.execute  (''' create table if not exists care_site 
                    (   cc_site_id integer primary key, 
                        care_site_name  varchar(255) not null,
                        location_id integer, 
                        care_site_source_value varchar(50),
                        foreign key (location_id) references location (location_id)
                    ) '''      
                ) 

#Insérer dans notre table
care_site.to_sql('care_site', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

In [35]:
# verifier que ça fonctionne 
conn = sqlite3.connect("omop.db")
cursor = conn.cursor()

# Lister toutes les tables
df_care_site = pd.read_sql_query("SELECT * FROM care_site;", conn)
print(df_care_site.head())

conn.close()


   cc_site_id                 care_site_name  care_site_source_value  \
0           1  l'Hôpital Privé des Peupliers               750300360   
1           2            Pharmacie Plaisance               750023772   

   location_id  
0            0  
1            0  
